# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [224]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [225]:
import numpy as np
from random import randint
import random
import pdb; 


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        '''ToDo: fix this function'''
        return 1.0 * (self.w ) / (self.C)  

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        '''ToDo: fix this function'''
        #pdb.set_trace()
        return -1 if (answer * self.f(x)) < 1 else 0



    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update
            '''ToDo: add w update with regularization'''

            self.w -= step * ( x * y * self.der_loss(x, y) + self.der_reg())
            
            # w_0 update
            '''ToDo: add w_0 update'''
            self.w0 -= step  * ( y * self.der_loss(x, y))

In [226]:
def custom_grid_search(Model, X_train, y_train, X_test, y_test):
  maximum = 0
  parametr = 0
  for k in range(1, 100):
    model = Model
    model.C = k
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    val = sum(predictions == y_test) / float(len(y_test))
    if (maximum < val):
      maximum = val
      parametr = k

  return maximum, k

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [227]:

np.random.seed(42)
random.seed(42)
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[-0.41072476  2.00505075  1.44069389  2.24268029  1.27190709  1.56290024
 -1.86694366 -0.33587805  0.04937335 -2.05935721 -0.92790213 -0.6932886
  1.5016486  -2.44691788 -1.62891772 -0.05627495  0.34800167  4.77037398
  2.3199871   0.31126646] -0.8289694298107237


In [228]:
predictions = model.predict(X_test)

In [229]:
print(predictions)

[0 1 0 ... 1 1 1]


In [230]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 1] 2000 991


In [231]:
print(len(predictions), sum(predictions))

2000 1186


In [232]:
print(sum(predictions == y_test) / float(len(y_test)))

0.6085


In [233]:
accuracy_max, C_max = custom_grid_search(model, X_train, y_train, X_test, y_test)

In [234]:

print(accuracy)

0.7265


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

Сравним c моделью из sklearn

In [221]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

clf = make_pipeline(StandardScaler(), LinearSVC(random_state=42, tol=1e-5))
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=42,
                           tol=1e-05, verbose=0))],
         verbose=False)

In [222]:
predictions = clf.predict(X_test)

In [223]:
print(sum(predictions == y_test) / float(len(y_test)))

0.7965


Видно, что sklearn LinearSVC показывает качество лучше чем MySVM.